In [1]:
!pip install arguebuf


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install openai==0.28


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import openai

In [5]:
# change the node type to edges

# Function to modify the graph
def modify_graph(graph):
    for node in graph.nodes:
        if graph.nodes[node]['label'] == 'Support':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'support' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='support')

        if graph.nodes[node]['label'] == 'Attack':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'attack' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='attack')

        if graph.nodes[node]['label'] == 'Rephrase':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'attack' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='rephrase')

def remove_support_attack_nodes(graph):
    nodes_to_remove = []

    for node in graph.nodes:
        if graph.nodes[node]['label'] == 'Support' or graph.nodes[node]['label'] == 'Attack' or graph.nodes[node]['label'] == 'Rephrase':
            nodes_to_remove.append(node)

    graph.remove_nodes_from(nodes_to_remove)

In [6]:
def chat(Messages):
    response = openai.ChatCompletion.create(
    model="gpt-4-1106-preview",
    messages=Messages
    )
    return response.choices[0].message.content

In [17]:
# Function to process children nodes
def process_nodes(graph, node, previous_summary=None):
        claim = graph.nodes[node]['label']
        children = list(graph.predecessors(node))

        support_premise = []
        attack_premise = []

        # Group premises based on support or attack
        for child in children:
            edge_type = graph.edges[child, node]['type']
            if edge_type == 'support':
                support_premise.append(graph.nodes[child]['label'])
            elif edge_type == 'attack':
                attack_premise.append(graph.nodes[child]['label'])

        # Generate text based on grouped premises using BART
        support_text = " ".join(support_premise)
        attack_text = " ".join(attack_premise)

        if support_text and attack_text:
            text_input = f'{claim} because, {support_text}. However,{attack_text}'
        elif support_text:
            text_input = f'{claim} because, {support_text}'
        elif attack_text:
            text_input = f'{claim} but, {attack_text}'


        if previous_summary:
            text_input = f"{previous_summary}{text_input}"

        print('text_input',text_input)

        Messages = [
                    {"role": "system", "content": "You are a text summarizer. Your task is to generate a concise summary of the provided debate within 90 words."},
                    {"role": "user", "content": text_input }
                ]
            
        summary = chat(Messages)
    
        print('summary',summary)

        summary_text = summary
        # Get the next level of nodes
        next_nodes = [child for child in children if graph.in_degree(child) > 0]
        for n in next_nodes:
          summary_text = process_nodes(graph, n, summary_text)

        return summary_text

In [18]:
import os
import arguebuf as ag
import networkx as nx
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# function to porcess arg graph and create bart divide conquer summaries
def create_gpt4_divideconquer_summaries(input_folder, output_folder):

    # Iterate through files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_file_path = os.path.join(input_folder, filename)
            graph = ag.load.file(input_file_path)
            ngraph = ag.dump.networkx(graph)

            # Call the function to modify the graph
            modify_graph(ngraph)
            # Call the function to remove support and attack nodes
            remove_support_attack_nodes(ngraph)

            # Start processing from the root node (claim)
            source_node = [node for node in ngraph.nodes() if len(list(ngraph.successors(node))) == 0][0]
            final_summary = process_nodes(ngraph, source_node)

            # write the concatenated result to a text file in the output folder
            output_file_path = os.path.join(output_folder, filename)
            with open(output_file_path, 'w', encoding="utf8") as output_file:
                output_file.write(final_summary)

In [19]:
# input and output folder paths
# input - arg graphs
# output - gpt4 summaries - using divide and conquer.
# Replace the below with desired input and output path

#input_folder_path = 'C:/Users/Nila/Documents/Main Scripts_25.12.2023/Jupyter Notebooks/complete testing/llm_input/arg_graphs/'
#output_folder_path = 'C:/Users/Nila/Documents/Main Scripts_25.12.2023/Jupyter Notebooks/complete testing/llm_output/BART/divide_conquer'

input_folder_path = "C:/Users/Nila/Documents/Project/benchmark/data/kialo-nilesc/sample-test/input/"
output_folder_path = "C:/Users/Nila/Documents/Project/benchmark/data/kialo-nilesc/sample-test/output/"

create_gpt4_divideconquer_summaries(input_folder_path, output_folder_path)

text_input A child's primary carer should receive a wage from the government until the child enters primary school or some other form of care. because, Such a measure may solve the challenge of low population growth confronting many nations. Raising a child is just as much, if not more, work than jobs which traditionally receive economic reward. Child raising benefits society just as much as any other job and therefore should receive economic recognition.. However,The government may interfere more in parenting practices if the parent essentially becomes an employee. The primary caregiver, usually the parent, chose to have a kid. Society should not be obligated to pay them to deal with their choices. Child raising is the responsibility of the family. People should have children for love, not money. Such a measure may not be economically viable.
summary The debate revolves around whether primary caregivers should be paid by the government until a child starts school as a solution to low 